In [1]:
!pip install pandas requests scikit-learn xgboost matplotlib seaborn

import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# New Section

In [11]:
from google.colab import files
import pandas as pd

# Upload CSVs
uploaded = files.upload()

# Load into DataFrames
races = pd.read_csv("races.csv")
results = pd.read_csv("results.csv")
drivers = pd.read_csv("drivers.csv")
constructors = pd.read_csv("constructors.csv")
circuits = pd.read_csv("circuits.csv")


Saving circuits.csv to circuits (1).csv
Saving constructor_results.csv to constructor_results (1).csv
Saving constructor_standings.csv to constructor_standings (1).csv
Saving constructors.csv to constructors (1).csv
Saving driver_standings.csv to driver_standings (1).csv
Saving drivers.csv to drivers (1).csv
Saving lap_times.csv to lap_times (1).csv
Saving pit_stops.csv to pit_stops (1).csv
Saving qualifying.csv to qualifying (1).csv
Saving races.csv to races (1).csv
Saving results.csv to results (1).csv
Saving seasons.csv to seasons (1).csv
Saving sprint_results.csv to sprint_results (1).csv
Saving status.csv to status (1).csv


In [13]:
# Drop URL columns before merge to avoid conflict
drivers.drop(columns=["url"], inplace=True)
constructors.drop(columns=["url"], inplace=True)
circuits.drop(columns=["url"], inplace=True)
races.drop(columns=["url"], inplace=True)

# Merge everything cleanly
df = results.merge(races, on="raceId") \
            .merge(drivers, on="driverId") \
            .merge(constructors, on="constructorId") \
            .merge(circuits, on="circuitId")

# Create full driver name
df["Driver"] = df["forename"] + " " + df["surname"]

# Select modeling columns
df_model = df[[
    "Driver",              # full name
    "name_x",              # race name
    "name_y",              # constructor name
    "name",                # circuit name
    "grid",
    "positionOrder",       # actual finishing position
    "date"
]]

# Rename for clarity
df_model.columns = ["Driver", "Race", "Constructor", "Circuit", "Grid", "Position", "Date"]

# Create target variable
df_model["Winner"] = (df_model["Position"] == 1).astype(int)

df_model.head()



/tmp/ipython-input-1121411193.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model["Winner"] = (df_model["Position"] == 1).astype(int)


,Driver,Race,Constructor,Circuit,Grid,Position,Date,Winner
0,Lewis Hamilton,Australian Grand Prix,McLaren,Albert Park Grand Prix Circuit,1,1,2008-03-16,1
1,Nick Heidfeld,Australian Grand Prix,BMW Sauber,Albert Park Grand Prix Circuit,5,2,2008-03-16,0
2,Nico Rosberg,Australian Grand Prix,Williams,Albert Park Grand Prix Circuit,7,3,2008-03-16,0
3,Fernando Alonso,Australian Grand Prix,Renault,Albert Park Grand Prix Circuit,11,4,2008-03-16,0
4,Heikki Kovalainen,Australian Grand Prix,McLaren,Albert Park Grand Prix Circuit,3,5,2008-03-16,0


In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode categorical variables
le_driver = LabelEncoder()
le_constructor = LabelEncoder()
le_circuit = LabelEncoder()

df_model['Driver_enc'] = le_driver.fit_transform(df_model['Driver'])
df_model['Constructor_enc'] = le_constructor.fit_transform(df_model['Constructor'])
df_model['Circuit_enc'] = le_circuit.fit_transform(df_model['Circuit'])

# Final feature set
features = ['Grid', 'Driver_enc', 'Constructor_enc', 'Circuit_enc']
X = df_model[features]
y = df_model['Winner']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


/tmp/ipython-input-2871895084.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Driver_enc'] = le_driver.fit_transform(df_model['Driver'])
/tmp/ipython-input-2871895084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Constructor_enc'] = le_constructor.fit_transform(df_model['Constructor'])
/tmp/ipython-input-2871895084.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train model
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict
preds = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))


Accuracy: 0.9570254110612855

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      5124
           1       0.49      0.17      0.25       228

    accuracy                           0.96      5352
   macro avg       0.73      0.58      0.62      5352
weighted avg       0.94      0.96      0.95      5352



In [16]:
import joblib

joblib.dump(model, "f1_winner_model.pkl")
joblib.dump(le_driver, "driver_encoder.pkl")
joblib.dump(le_constructor, "constructor_encoder.pkl")
joblib.dump(le_circuit, "circuit_encoder.pkl")


['circuit_encoder.pkl']